In [2]:
from pyspark.sql import Row, SparkSession
spark = SparkSession.builder.appName("movie").getOrCreate()

your 131072x1 screen size is bogus. expect trouble


23/02/19 12:08:47 WARN Utils: Your hostname, DESKTOP-60CN0SA resolves to a loopback address: 127.0.1.1; using 172.29.46.194 instead (on interface eth0)
23/02/19 12:08:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/19 12:08:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/02/19 12:08:50 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [15]:
# RATINGS_FILE = "../data/mllib/sample_movielens_data.txt"
RATINGS_FILE = "ml-latest-small/ratings.csv"
MOVIES_FILE = "ml-latest-small/movies.csv"

In [74]:
import pandas as pd
movies = pd.read_csv(MOVIES_FILE)
movies[:5]

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [75]:
movies_df = spark.createDataFrame(movies)
movies_df.show(5)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows



In [24]:
with open(RATINGS_FILE, 'rt') as f:
    ratings = f.readlines()
ratings[:5]

['userId,movieId,rating,timestamp\n',
 '1,1,4.0,964982703\n',
 '1,3,4.0,964981247\n',
 '1,6,4.0,964982224\n',
 '1,47,5.0,964983815\n']

In [43]:
df = spark.read.option('header', 'true').csv(RATINGS_FILE, inferSchema=True)
print(df.count())
display(df.dtypes)
df.show(5)

100836


[('userId', 'int'),
 ('movieId', 'int'),
 ('rating', 'double'),
 ('timestamp', 'int')]

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows



In [37]:
from pyspark.ml.recommendation import ALS

als = ALS(maxIter=3, userCol='userId', itemCol='movieId', ratingCol='rating', coldStartStrategy='drop')


In [45]:

(train, test) = df.randomSplit([.8, .2])

In [46]:
model = als.fit(train)

23/02/19 12:41:05 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [50]:
from pyspark.ml.evaluation import RegressionEvaluator
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction' )

In [55]:
evaluator.evaluate(predictions)

0.9124561115814143

In [68]:
import pyspark.sql.functions as F

In [81]:
model.recommendForAllUsers(1).dtypes

[('userId', 'int'),
 ('recommendations', 'array<struct<movieId:int,rating:float>>')]

In [84]:
model.recommendForAllUsers(1).select(F.col("recommendations[].movieId")).show(1)

AnalysisException: Column '`recommendations[]`.movieId' does not exist. Did you mean one of the following? [recommendations, userId];
'Project ['recommendations[].movieId]
+- Project [id#4648 AS userId#4652, cast(recommendations#4649 as array<struct<movieId:int,rating:float>>) AS recommendations#4653]
   +- Project [key#4642 AS id#4648, TopByKeyAggregator(scala.Tuple3)#4647 AS recommendations#4649]
      +- Aggregate [value#4634], [value#4634 AS key#4642, topbykeyaggregator(org.apache.spark.ml.recommendation.TopByKeyAggregator@515c3340, Some(newInstance(class scala.Tuple3)), Some(class scala.Tuple3), Some(StructType(StructField(_1,IntegerType,false),StructField(_2,IntegerType,false),StructField(_3,FloatType,false))), encodeusingserializer(input[0, java.lang.Object, true], true), decodeusingserializer(input[0, binary, true], org.apache.spark.util.BoundedPriorityQueue, true), mapobjects(lambdavariable(MapObject, ObjectType(class scala.Tuple2), true, 451), if (isnull(lambdavariable(MapObj

In [94]:
model.recommendForAllUsers(1).select(F.element_at(F.col("recommendations.movieId"),1).alias('movieId')).join(movies_df, on='movieId').show()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|    720|Wallace & Gromit:...|Adventure|Animati...|
|    720|Wallace & Gromit:...|Adventure|Animati...|
|    720|Wallace & Gromit:...|Adventure|Animati...|
|   1280|Raise the Red Lan...|               Drama|
|   1241|Dead Alive (Brain...|Comedy|Fantasy|Ho...|
|    932|Affair to Remembe...|       Drama|Romance|
|    932|Affair to Remembe...|       Drama|Romance|
|    417|    Barcelona (1994)|      Comedy|Romance|
|    232|Eat Drink Man Wom...|Comedy|Drama|Romance|
|    232|Eat Drink Man Wom...|Comedy|Drama|Romance|
|    446|Farewell My Concu...|       Drama|Romance|
|     28|   Persuasion (1995)|       Drama|Romance|
|     28|   Persuasion (1995)|       Drama|Romance|
|     28|   Persuasion (1995)|       Drama|Romance|
|     28|   Persuasion (1995)|       Drama|Romance|
|     28|   Persuasion (1995)|       Drama|Romance|
|     28|   

In [67]:
model.recommendForAllUsers(1).dtypes

[('userId', 'int'),
 ('recommendations', 'array<struct<movieId:int,rating:float>>')]